In [1]:
import os
import pandas as pd
from datetime import datetime
import numpy as np
base_dir = ""

In [2]:
from torch.utils.data import Dataset, DataLoader
class NumpyDataset(Dataset):
    def __init__(self, ds, lag):
        self.x_cat = ds[:,0:3]
        self.x_time = np.expand_dims(ds[:,3:-1], -1)
        self.y = ds[:,-1]
    def __len__(self): 
        return len(self.y)
    def __getitem__(self, idx):
        return self.x_cat[idx], self.x_time[idx], self.y[idx]

In [4]:
%cd

/root


In [15]:
import pickle
train_ds = pickle.load(open( os.path.join(base_dir,"train_10.pkl"), "rb" ) )
test_ds = pickle.load(open( os.path.join(base_dir,"test_10.pkl"), "rb" ) )

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_ds[:,:-1] = scaler.fit_transform(train_ds[:,:-1])
test_ds[:,:-1] = scaler.transform(test_ds[:,:-1])

In [7]:
train_ds = NumpyDataset(train_ds, lag=10)
test_ds = NumpyDataset(test_ds, lag=10)

batch_size = 200

train_loader = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last = True)
test_loader = DataLoader(test_ds, shuffle=True, batch_size=batch_size, drop_last = True)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):   
    def __init__(self, lag = 10, batch_size = 200):
        super(Net, self).__init__()
        self.hidden_size = 10
        self.enlstm1 = nn.LSTM(1, hidden_size= self.hidden_size, num_layers = 2, batch_first=True)
        self.delstm1 = nn.LSTM(self.hidden_size, hidden_size = self.hidden_size, num_layers = 2, batch_first = True)
        self.dropout = nn.Dropout(0.25)
        self.linear1 = nn.Linear(self.hidden_size * (lag-1), 32)
        
        self.linear_layers = nn.Sequential(
            nn.Linear(3, 16),
            nn.Linear(16,32),
            nn.Dropout(0.25),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(64,32),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            nn.Linear(32,1),
        )

    # Defining the forward pass    
    def forward(self, x1, x2):
        x1 = self.linear_layers(x1)
        x2, h2 = self.enlstm1(x2)
        x2 = x2.contiguous().view(200,-1, self.hidden_size)
        x2, _ = self.delstm1(x2, h2)
        x2 = x2.contiguous().view(200,-1)
        x2 = self.linear1(x2)
        x2 = self.dropout(x2)
        
        x = torch.cat([x1, x2], axis = 1)
        x = self.fc(x)
        return x

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Net(lag=10)
model.to(device)
lr=0.05
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 30
print(model)

Net(
  (enlstm1): LSTM(1, 10, num_layers=2, batch_first=True)
  (delstm1): LSTM(10, 10, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (linear1): Linear(in_features=90, out_features=32, bias=True)
  (linear_layers): Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=32, bias=True)
    (2): Dropout(p=0.25, inplace=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [11]:
model.train()
loss_min = np.inf
for i in range(epochs):
    for batch_idx, (x1, x2, y) in enumerate(train_loader):
        x1, x2, y = x1.to(device).float(), x2.to(device).float(), y.to(device).float()
        model.zero_grad()
        output = model(x1, x2)
        loss = criterion(output.flatten(), y)
        loss.backward()
        optimizer.step()
        if batch_idx == 0:
            test_loss = []
            for xx1, xx2, yy in test_loader:
                xx1, xx2, yy = xx1.to(device).float(), xx2.to(device).float(), yy.to(device).float()
                model.zero_grad()
                yyout = model(xx1, xx2)
                test_loss.append(criterion(yyout.flatten(), yy).detach().cpu().numpy())
            print("Epoch: {}, Loss: {:.2f}, Val Loss: {:.2f}".format(i, loss, np.mean(test_loss)))

Epoch: 0, Loss: 1.39, Val Loss: 26.33
Epoch: 1, Loss: 0.47, Val Loss: 23.90
Epoch: 2, Loss: 0.69, Val Loss: 24.29
Epoch: 3, Loss: 6.31, Val Loss: 24.21
Epoch: 4, Loss: 0.36, Val Loss: 24.23
Epoch: 5, Loss: 2.00, Val Loss: 24.22
Epoch: 6, Loss: 1.60, Val Loss: 24.23
Epoch: 7, Loss: 0.82, Val Loss: 24.27
Epoch: 8, Loss: 0.40, Val Loss: 24.21


KeyboardInterrupt: ignored

In [ ]:
# https://www.kaggle.com/karanjakhar/simple-and-easy-aprroach-using-lstm